# 7.4 Restrictions on parameters

In [1]:
# install dependencies
%pip install -q amplpy

from amplpy import AMPL, ampl_notebook

ampl = ampl_notebook(
    modules=['highs'],  # modules to install
    license_uuid='default',  # license to use
)  # instantiate AMPL object and register magics


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


If T is intended to represent the number of weeks in a multiperiod model, it should be
an integer and greater than 1. By including these conditions in T's declaration,
```
param T > 1 integer;
```
you instruct AMPL to reject your data if you inadvertently set `T` to 1:
```
error processing param T:
	failed check: param T = 1
       is not > 1;
```
or to 2.5:
```
error processing param T:
	failed check: param T = 2.5
		is not an integer;
```
AMPL will not send your problem instance to a solver as long as any errors of this kind
remain.

In the declaration of an indexed collection of parameters, a simple restriction such as
integer or >= 0 applies to every parameter defined. Our examples often use this
option to specify that vectors and arrays are nonnegative:
```
param demand {DEST,PROD} >= 0;
```
If you include dummy indices in the indexing expression, however, you can use them to
specify a different restriction for each parameter:
```
param f_min {FOOD} >= 0;
param f_max {j in FOOD} >= f_min[j];
```

The effect of these declarations is to define a pair of parameters
`f_max[j] >= f_min[j]` for every `j` in the set `FOOD`.

A restriction phrase for a parameter declaration may be the word integer or
binary or a comparison operator followed by an arithmetic expression. While
integer restricts a parameter to integral (whole-number) values, binary restricts it to
zero or one. The arithmetic expression may refer to sets and parameters previously
defined in the model, and to dummy indices defined by the current declaration. There
may be several restriction phrases in the same declaration, in which case they may
optionally be separated by commas.

In special circumstances, a restriction phrase may even refer to the parameter in
whose declaration it appears. Some multiperiod production models, for example, are
defined in terms of a parameter `cumulative_market[p,t]` that represents the
cumulative demand for product `p` in weeks 1 through `t`. Since cumulative demand does
not decrease, you might try to write a restriction phrase like this:
```
param cumulative_market {p in PROD, t in 1..T}
  >= cumulative_market[p,t-1];   # ERROR
```
For the parameters `cumulative_market[p,1]`, however, the restriction phrase will
refer to `cumulative_market[p,0]`, which is undefined; AMPL will reject the declaration
with an error message. What you need here again is a conditional expression that
handles the first period specially:
```
param cumulative_market {p in PROD, t in 1..T}
  >= if t = 1 then 0 else cumulative_market[p,t-1];
```
The same thing could be written a little more compactly as
```
param cumulative_market {p in PROD, t in 1..T}
  >= if t > 1 then cumulative_market[p,t-1];
```
since "else 0" is assumed. Almost always, some form of if-then-else expression
is needed to make this kind of self-reference possible.

As you might suspect from this last example, sometimes it is desirable to place a more
complex restriction on the model's data than can be expressed by a restriction phrase
within a declaration. This is the purpose of the `check` statement. For example, in the
transportation model of [Figure 3-1a](./3_2_an_AMPL_model_for_the_transportation_problem.ipynb#fig-3-1a), total supply must equal total demand:
```
check: sum {i in ORIG} supply[i] = sum {j in DEST} demand[j];
```
The multicommodity version, in [Figure 4-1](../04/4_1_a_multicommodity_transportation_model.ipynb#fig-4-1), uses an indexed check to say that total supply must equal total demand for each product:
```
check {p in PROD}:
  sum {i in ORIG} supply[i,p] = sum {j in DEST} demand[j,p];
```
Here the restriction is tested once for each member `p` of `PROD`. If the check fails for any
member, AMPL prints an error message and rejects all of the data.

You can think of the `check` statement as specifying a kind of constraint, but only on
the data. The restriction clause is a logical expression, which may use any previously
defined sets and parameters as well as dummy indices defined in the statement's indexing
expression. After the data values have been read, the logical expression must evaluate to
true; if an indexing expression has been specified, the logical expression is evaluated separately
for each assignment of set members to the dummy indices, and must be true for
each.

We strongly recommend the use of restriction phrases and `check` statements to validate
a model's data. These features will help you to catch data errors at an early stage,
when they are easy to fix. Data errors not caught will, at best, cause errors in the generation
of the variables and constraints, so that you will get some kind of error message from
AMPL. In other cases, data errors lead to the generation of an incorrect linear program. If
you are fortunate, the incorrect LP will have a meaningless optimal solution, so that —
possibly after a good deal of effort — you will be able to work backward to find the error
in the data. At worst, the incorrect LP will have a plausible solution, and the error will go
undetected.
